In [86]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [87]:
combos, costs = gp.multidict(
    {
        ("A", "1"): 15,
        ("A", "2"): 13.8,
        ("A", "3"): 12.5,
        ("A", "4"): 11,
        ("A", "5"): 14.3,
        ("B", "1"): 14.5,
        ("B", "2"): 14,
        ("B", "3"): 13.2,
        ("B", "4"): 10.5,
        ("B", "5"): 15,
        ("C", "1"): 13.8,
        ("C", "2"): 13,
        ("C", "3"): 12.8,
        ("C", "4"): 11.3,
        ("C", "5"): 14.6,
        ("D", "1"): 14.7,
        ("D", "2"): 13.6,
        ("D", "3"): 13,
        ("D", "4"): 11.6,
        ("D", "5"): 14,
    }
)

In [88]:
combos_df = pd.DataFrame(combos)
companies = [i for i in combos_df[0].unique()]
shops = [i for i in combos_df[1].unique()]

In [89]:
m = gp.Model()

In [90]:
x = m.addVars(combos, vtype=GRB.BINARY, name="x")

In [91]:
m.setObjective(x.prod(costs))

In [92]:
m.addConstrs((x.sum(company, "*") <= 2 for company in companies), name="公司限制")

{'A': <gurobi.Constr *Awaiting Model Update*>,
 'B': <gurobi.Constr *Awaiting Model Update*>,
 'C': <gurobi.Constr *Awaiting Model Update*>,
 'D': <gurobi.Constr *Awaiting Model Update*>}

In [93]:
m.addConstrs((x.sum("*", shop) == 1 for shop in shops), name="门店限制")

{'1': <gurobi.Constr *Awaiting Model Update*>,
 '2': <gurobi.Constr *Awaiting Model Update*>,
 '3': <gurobi.Constr *Awaiting Model Update*>,
 '4': <gurobi.Constr *Awaiting Model Update*>,
 '5': <gurobi.Constr *Awaiting Model Update*>}

In [94]:
m.write("IP.lp")

In [95]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x1753dfe0
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 65.0000000
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 63.8000000

Root relaxation: cutoff, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        63.80000   63.80000  0.00%     -    0s

Exp

In [96]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 63.8
Optimal solution:
x[A,3] = 1.0
x[B,4] = 1.0
x[C,1] = 1.0
x[C,2] = 1.0
x[D,5] = 1.0
